# Speak with your Graph
### Neo4j Graph
The Neo4j Graph integration is a wrapper for the Neo4j Python driver. It allows querying and updating the Neo4j database in a simplified manner from LangChain. Many integrations allow you to use the Neo4j Graph as a source of data for LangChain.

### CypherQAChain
The CypherQAChain is a LangChain component that allows you to interact with a Neo4j graph database in natural language. Using an LLM and the graph schema it translates the user question into a Cypher query, executes it against the graph and uses the returned context information and the original question with a second LLM to generate a natural language response.

- https://api.python.langchain.com/en/latest/chains/langchain.chains.graph_qa.cypher.GraphCypherQAChain.html

# Neo4jVector
The Neo4j Vector integration supports a number of operations

- create vector from langchain documents
- query vector
- query vector with additional graph retrieval Cypher query
- construct vector instance from existing graph data
- hybrid search
- metadata filtering

### INstall exampe

```
pip install langchain langchain-community langchain-neo4j
pip install langchain-openai tiktoken
pip install neo4j
```

# MCP Model Context Protocol
https://www.anthropic.com/news/model-context-protocol

### https://modelcontextprotocol.io/introduction
The Model Context Protocol allows applications to provide context for LLMs in a standardized way, separating the concerns of providing context from the actual LLM interaction. This Python SDK implements the full MCP specification, making it easy to:

- Build MCP clients that can connect to any MCP server
- Create MCP servers that expose resources, prompts and tools
- Use standard transports like stdio and SSE
- Handle all MCP protocol messages and lifecycle events

## links
- https://github.com/modelcontextprotocol/python-sdk
- https://neo4j.com/blog/developer/knowledge-graphs-claude-neo4j-mcp/
- https://github.com/neo4j-contrib/mcp-neo4j/tree/main
- https://pypi.org/project/mcp-neo4j-cypher/
- 
# CLaude Desktop via MCP

### Install 
https://claude.ai/download

```
linux/mac
curl -LsSf https://astral.sh/uv/install.sh | sh

windows
powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"

"neo4j": {
    "command": "/Users/<username>/.local/bin/uvx", # full path to your 'uv' executable
    "args": [
        "mcp-neo4j-cypher",
        "--db-url",
        "bolt://localhost",
        "--username",
        "neo4j",
        "--password",
        "password"
    ]
}
```


In [1]:
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_openai import ChatOpenAI

In [80]:
import logging
import warnings
from dotenv import load_dotenv
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import pprint

In [114]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [3]:
status = load_dotenv(".env")
config = dotenv_values(".env")

In [115]:
llm = ChatOpenAI(
    model="gpt-4.1",
    openai_api_key=config["OPENAI_API_KEY"],
    temperature=0,
    max_tokens=8192

)
embeddings = OpenAIEmbeddings()

In [83]:
uri = "bolt://127.0.0.1:7687"
username = "neo4j"
password = "" 

# Neo4jVector
The Neo4j Vector integration supports a number of operations

- create vector from langchain documents
- query vector
- query vector with additional graph retrieval Cypher query
- construct vector instance from existing graph data
- hybrid search
- metadata filtering

In [117]:
loader = TextLoader("state_of_the_union.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [122]:
#documents[0]

In [139]:
docs[0], len(docs)

(Document(metadata={'source': 'state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determin

In [119]:
db = Neo4jVector.from_documents(
    docs, embeddings, url=uri, username=username, password=password
)

query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [120]:
docs_with_score

[(Document(metadata={'source': 'state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
  0.902252197265625),
 (Document(metadata={'source': 'state_of_the_union.txt'}, page_content='A former top litigator in private pra

# Neo4j Graph
The Neo4j Graph integration is a wrapper for the Neo4j Python driver. It allows querying and updating the Neo4j database in a simplified manner from LangChain. Many integrations allow you to use the Neo4j Graph as a source of data for LangChain.

In [140]:
graph = Neo4jGraph(url=uri, username=username, password=password)

In [141]:
#  Neo4jGraph
query = """
MATCH (v:Visitor_this_year {BadgeId:$badgeID})
WITH v.JobTitle AS jt, v.job_role AS jr, v.what_type_does_your_practice_specialise_in AS spec
MATCH (v2:Visitor_this_year)
WHERE v2.assist_year_before = "1"
  AND (
    toLower(v2.JobTitle) = toLower(jt) OR
    toLower(v2.job_role) = toLower(jr) OR
    toLower(v2.what_type_does_your_practice_specialise_in) = toLower(spec)
  )
WITH v2
LIMIT 3
OPTIONAL MATCH (v2)-[:Same_Visitor]->(vlva:Visitor_last_year_lva)
OPTIONAL MATCH (v2)-[:Same_Visitor]->(vbva:Visitor_last_year_bva)
OPTIONAL MATCH (vlva)-[:attended_session]->(s1:Sessions_past_year)
OPTIONAL MATCH (vbva)-[:attended_session]->(s2:Sessions_past_year)
RETURN v2.BadgeId AS this_year_BadgeId, collect(DISTINCT s1) + collect(DISTINCT s2) AS sessions_last_year
"""

In [142]:
resp = graph.query(query, params={"badgeID": "VPH826J"})

In [144]:
#resp

# CypherQAChain
The CypherQAChain is a LangChain component that allows you to interact with a Neo4j graph database in natural language. Using an LLM and the graph schema it translates the user question into a Cypher query, executes it against the graph and uses the returned context information and the original question with a second LLM to generate a natural language response.

In [145]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps= True
)

In [146]:
chain.invoke("How many Sessions this year have a connection Has_stream to the Stream nursing?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Sessions_this_year)-[:HAS_STREAM]->(st:Stream {stream: "nursing"})
RETURN count(s) AS session_count
Full Context:
[{'session_count': 12}]

> Finished chain.


{'query': 'How many Sessions this year have a connection Has_stream to the Stream nursing?',
 'result': 'There are 12 sessions this year that have a connection Has_stream to the Stream nursing.',
 'intermediate_steps': [{'query': 'MATCH (s:Sessions_this_year)-[:HAS_STREAM]->(st:Stream {stream: "nursing"})\nRETURN count(s) AS session_count'},
  {'context': [{'session_count': 12}]}]}

In [147]:
response= chain.invoke("find all the Visitors this year which  has as job title 'vet/vet surgeon' and  have been in the bva show last year?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visitor_this_year {JobTitle: 'vet/vet surgeon'})-[:Same_Visitor]->(:Visitor_last_year_bva)
RETURN v
Full Context:
[{'v': {'BadgeId_last_year_lva': 'NA', 'organisation_type': 'Corporate Group', 'Company': 'The Veterinary Clinic', 'Email': 'vivkw.chan@gmail.com', 'job_role': 'Vet/Vet Surgeon', 'ShowRef': 'BVA2025', 'what_type_does_your_practice_specialise_in': 'Small Animal', 'BadgeId': 'F5C5FRF', 'JobTitle': 'vet/vet surgeon', 'Source': 'BVABulletin', 'Email_domain': 'gmail.com', 'BadgeId_last_year_bva': 'RL88IR7', 'Days_since_registration': '146', 'Country': 'UK', 'assist_year_before': '1', 'BadgeType': 'Delegate'}}, {'v': {'BadgeId_last_year_lva': 'NA', 'organisation_type': 'Corporate Group', 'Company': 'Scarsdale Vets', 'Email': 'pam.p.oneill@googlemail.com', 'job_role': 'Vet/Vet Surgeon', 'ShowRef': 'BVA2025', 'what_type_does_your_practice_specialise_in': 'Mixed', 'BadgeId': '8I9T8GD', 'JobTitle': 'vet/vet surge

In [148]:
pprint.pprint(response.get("result"))

("The following visitors this year have the job title 'vet/vet surgeon' and "
 'have been in the BVA show last year: The Veterinary Clinic, Scarsdale Vets, '
 'Ark House Vets, PDSA Sheffield, Karen Casswell Ltd, PDSA, Dr Judy Scrine, St '
 'Davids Vets, None currently., Tower vet centre.')


In [149]:
# 764KTZS
response= chain.invoke(" find for  the Visitor this year with batchID F5C5FRF find 5 sessions this year which are in one of the stream connected using the  relationship job_to_stream")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visitor_this_year {BadgeId: "F5C5FRF"})-[:job_to_stream]->(s:Stream)<-[:HAS_STREAM]-(sess:Sessions_this_year)
RETURN sess
LIMIT 5
Full Context:
[{'sess': {'date': '2025-06-13', 'start_time': '09:00:00', 'sponsored_by': 'Not Sponsored', 'theatre__name': 'Clinical Theatre 1', 'synopsis_stripped': 'No Data', 'key_text': 'canineneuteringincontextwhowhenwhy', 'stream': 'Reproduction; Small Animal', 'end_time': '09:50:00', 'session_id': '49677', 'title': 'Canine neutering in context: Who? When? Why?', 'sponsored_session': 'False'}}, {'sess': {'date': '2025-06-13', 'start_time': '16:00:00', 'sponsored_by': 'Not Sponsored', 'theatre__name': 'Clinical Theatre 1', 'synopsis_stripped': 'No Data', 'key_text': 'canineliverdiseasethelatestmanagementrecommendations', 'stream': 'Internal Medicine; Internal Medicine', 'end_time': '16:50:00', 'session_id': '49683', 'title': 'Canine liver disease: the latest management recommendation

In [150]:
pprint.pprint(response.get("result"))

('Here are 5 sessions this year in streams connected via the job_to_stream '
 'relationship:\n'
 '\n'
 '1. Canine neutering in context: Who? When? Why? (Reproduction; Small '
 'Animal)\n'
 '2. Canine liver disease: the latest management recommendations (Internal '
 'Medicine; Internal Medicine)\n'
 '3. How to diagnose liver disease in practice (Internal Medicine)\n'
 '4. Solving the lame dog… one step at a time (Orthopaedics; Orthopaedics; '
 'Orthopaedics; Small Animal; Small Animal; Small Animal)\n'
 '5. Paws and peace - navigating end of life care for pets (Geriatric '
 'Medicine; Small Animal)')


In [151]:
response= chain.invoke("""
find for  the Visitor this year with batchID F5C5FRF find 10 sessions this year which are in one of the stream connected using the  relationship job_to_stream,
from the resulting sessions exclude those which contains Equine in the attribute stream
Bear in mind that the attribute stream in sessions is a string that can contains multiple Streams separated by ; ex. 'Orthopaedics; Orthopaedics; Orthopaedics; Small Animal; Small Animal; Small Animal' """)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visitor_this_year {BadgeId: "F5C5FRF"})-[:job_to_stream]->(s:Stream)<-[:HAS_STREAM]-(sess:Sessions_this_year)
WHERE NOT sess.stream CONTAINS "Equine"
RETURN sess
LIMIT 10
Full Context:
[{'sess': {'date': '2025-06-13', 'start_time': '09:00:00', 'sponsored_by': 'Not Sponsored', 'theatre__name': 'Clinical Theatre 1', 'synopsis_stripped': 'No Data', 'key_text': 'canineneuteringincontextwhowhenwhy', 'stream': 'Reproduction; Small Animal', 'end_time': '09:50:00', 'session_id': '49677', 'title': 'Canine neutering in context: Who? When? Why?', 'sponsored_session': 'False'}}, {'sess': {'date': '2025-06-13', 'start_time': '16:00:00', 'sponsored_by': 'Not Sponsored', 'theatre__name': 'Clinical Theatre 1', 'synopsis_stripped': 'No Data', 'key_text': 'canineliverdiseasethelatestmanagementrecommendations', 'stream': 'Internal Medicine; Internal Medicine', 'end_time': '16:50:00', 'session_id': '49683', 'title': 'Canine liver dise

In [152]:
pprint.pprint(response.get("result"))

('Here are 10 sessions this year in the relevant streams, excluding those with '
 '"Equine" in the stream attribute:\n'
 '\n'
 '1. Canine neutering in context: Who? When? Why?\n'
 '2. Canine liver disease: the latest management recommendations\n'
 '3. How to diagnose liver disease in practice\n'
 '4. Solving the lame dog… one step at a time\n'
 '5. Paws and peace - navigating end of life care for pets\n'
 "6. Cat friendly interactions - avoiding 'cat' astrophes at the clinic\n"
 "7. Improving our feline patients' hospital journey\n"
 '8. Why, when and how of BOAS assessment\n'
 '9. Where is the UK on bTB control?\n'
 '10. Preparing for Change: The CMA Investigation, where we are and possible '
 'implications for the profession\n'
 '\n'
 'None of these sessions contain "Equine" in their stream attribute.')


In [156]:
response= chain.invoke("""
for batchID UEXNXI4 
find 3 visitors this year which assist the year before is 1  with  equal or similar: JobTitle,  job_role and what_type_does_your_practice_specialise_in
and for the 3 visitors you find return all sessions they visited last year and the batchID this year

 """)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visitor_this_year {BadgeId: "UEXNXI4"})
WITH v.JobTitle AS jt, v.job_role AS jr, v.what_type_does_your_practice_specialise_in AS spec
MATCH (v2:Visitor_this_year)
WHERE v2.assist_year_before = "1"
  AND (
    toLower(v2.JobTitle) = toLower(jt) OR toLower(v2.job_role) = toLower(jr) OR toLower(v2.what_type_does_your_practice_specialise_in) = toLower(spec)
  )
WITH v2 LIMIT 3
OPTIONAL MATCH (v2)-[:Same_Visitor]->(vlb:Visitor_last_year_bva)
OPTIONAL MATCH (v2)-[:Same_Visitor]->(vll:Visitor_last_year_lva)
OPTIONAL MATCH (vlb)-[:attended_session]->(s1:Sessions_past_year)
OPTIONAL MATCH (vll)-[:attended_session]->(s2:Sessions_past_year)
RETURN v2.BadgeId AS this_year_BadgeId, collect(DISTINCT s1) + collect(DISTINCT s2) AS sessions_last_year
Full Context:
[{'this_year_BadgeId': 'S2SHRH4', 'sessions_last_year': [{'date': '15/11/2024', 'start_time': '10:35:00', 'sponsored_by': 'Tonisity', 'theatre__name': 'RVC Clinical Theat

In [157]:
print(response.get("result"))

I don't know the answer.


In [158]:
pprint.pprint(response.get("intermediate_steps")[1]['context'])

[{'sessions_last_year': [{'date': '15/11/2024',
                          'end_time': '11:25:00',
                          'key_text': 'managingthepatientwithgdvbeforeduringandaftersurgery',
                          'session_id': '42370',
                          'sponsored_by': 'Tonisity',
                          'sponsored_session': 'False',
                          'start_time': '10:35:00',
                          'stream': 'No Data',
                          'synopsis_stripped': 'Gastric dilatation volvulus '
                                               '(GDV) represents a dire '
                                               'surgical emergency '
                                               'necessitating immediate '
                                               'recognition and intervention. '
                                               'Affected patients typically '
                                               'present in a severely '
                         